In [2]:
import numpy as np
from SALib.sample import morris as morris_sample
from SALib.analyze import morris as morris_analyze
import math

# Define your model function here
def model_function(x):
    y = [1*math.sin(x[0]),2*math.sin(x[1]),3*math.sin(x[2]),4*math.sin(x[3]),5*math.sin(x[4]),6*math.sin(x[5]),7*math.sin(x[6])]
    return y

# Define the number of samples and levels
n_samples = 1000
n_levels = 4

# Define the ranges for each parameter
problem = {
    'num_vars': 7,
    'names': ['param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'param7'],
    'bounds': [[0, 1]] * 7  # Assuming all parameters have a range from 0 to 1
}

# Generate the Morris samples
param_values = morris_sample.sample(problem, n_samples, num_levels=n_levels)

# Evaluate the model for the generated samples
output_values = model_function(param_values)

# Perform the Morris analysis
morris_results = morris_analyze.analyze(problem, param_values, output_values, num_levels=n_levels, conf_level=0.95, print_to_console=False)

# Extract the mu and sigma values
mu = morris_results['mu']
sigma = morris_results['sigma']

# Print the results
for i, name in enumerate(problem['names']):
    print(f'{name}: mu={mu[i]}, sigma={sigma[i]}')

TypeError: only size-1 arrays can be converted to Python scalars

In [4]:
def morris_indices(model, inputs, n_samples=100, n_levels=4):
    num_inputs = inputs.shape[1]
    num_outputs = model(inputs[0:1]).shape[1]
    num_combinations = num_inputs * (n_levels - 1)

    # Generate the perturbation matrix
    perturbation_matrix = np.zeros((n_samples, num_combinations))
    for i in range(num_combinations):
        perturbation_matrix[:, i] = np.random.choice([-1, 1], n_samples)

    # Compute the baseline predictions
    baseline_preds = model(inputs)

    # Initialize arrays for storing the differences and indices
    differences = np.zeros((num_combinations, n_samples))
    indices = np.zeros((num_combinations,))

    # Perturb each input parameter and compute the differences
    for i in range(num_combinations):
        perturbed_inputs = np.tile(inputs, (n_samples, 1))
        perturbed_inputs[:, i // (n_levels - 1)] += perturbation_matrix[:, i]
        perturbed_preds = model(perturbed_inputs)
        differences[i] = np.abs(perturbed_preds - baseline_preds).mean(axis=1)

    # Compute the Morris indexes (Mu and Sigma)
    indices = differences.mean(axis=1) / differences.sum()
    sigma = differences.std(axis=1) / differences.sum()

    return indices, sigma

# Example usage
def model(x):
    y = [1*math.sin(x[0]),2*math.sin(x[1]),3*math.sin(x[2]),4*math.sin(x[3]),5*math.sin(x[4]),6*math.sin(x[5]),7*math.sin(x[6])]
    return y

# Generate sample inputs
np.random.seed(42)
inputs = np.random.rand(100, 7)

# Compute Morris indexes
mu, sigma = morris_indices(model, inputs)

# Print results
for i in range(len(mu)):
    print(f"Mu[{i}]: {mu[i]}")
    print(f"Sigma[{i}]: {sigma[i]}")

TypeError: only size-1 arrays can be converted to Python scalars

In [13]:
import pyvista as pv

# Read the file
filename = r'D:\Andrea\Flow\SensitivityAnalisys\meshSA.vtu'
mesh = pv.read(filename)

# Get the Z coordinates
z_coords = mesh.points[:, 2]

# Multiply the Z coordinates by 100
new_z_coords = z_coords * 100

# Update the mesh with the new Z coordinates
mesh.points[:, 2] = new_z_coords

v_data = mesh.get_array('v')

modified_v_data  = np.copy(v_data)

# Multiply the 'v' component by 100
modified_v_data[:,2]=v_data[:,2] * 100

# Update the 'v' component with the modified values
mesh.point_data['v'] = modified_v_data

# Save the modified mesh
output_filename = r'D:\Andrea\Flow\SensitivityAnalisys\100xMeshSA.vtu'
mesh.save(output_filename)